![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Final Project: Booking - Attractions Reviews Scrapping

## Scraping with Selenium + Beautiful Soup; Regex with html_element text preprocess

Selemnium is able to simulate human-like action in webpage.<br>
Beautiful Soup is able to fetch entire or selected html page, we directly operate search and select text on that.<br>
We use Regex to operate text preprocess, our case is regex on html_element<br>

In [1]:
#import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
#import requests

import os
import pandas as pd
from datetime import datetime
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    #profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,firefox_binary_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    options.binary_location = firefox_binary_path #binary location
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

In [14]:
# start selenium
dfolder='C:/Users/surface/DSDM/Semesters/semester2/text mining/Final Project/downloads/'
geko_path='C:/Users/surface/DSDM/Semesters/semester2/text mining/geckodriver.exe'
# Barcelona
#Sagrada-Familia-top1
link='https://www.booking.com/attractions/es/prbspnfdkbkw-admission-to-sagrada-familia.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-372490&date=2024-03-18&start_time=09%3A45&ticket_type=OFo8ZsK8ZAZw'
save_file_path ="/Barcelona_Attraction_Reviews_Booking-top1.csv"
# Barcelona
#Park Guell Admission Ticket-top2
link = 'https://www.booking.com/attractions/es/prp9at1egmyc-park-guell-admission-ticket.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-372490&date=2024-03-19&start_time=09%3A30&ticket_type=OFmQYxiImuT3'
save_file_path ="/Barcelona_Attraction_Reviews_Booking-top2.csv"
#Venice Islands Day Trip: Murano, Burano & Torcello - top 1
#link = 'https://www.booking.com/attractions/it/pr02wwrbrveo-venice-islands-day-trip-murano-burano-torcello.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-132007&date=2024-03-21&start_time=11%3A00&ticket_type=OFgcmgG5EVNi'
#save_file_path = "/Venice_Attraction_Reviews_Booking-top1.csv"
#Venice: Four Seasons Concert in the Vivaldi Church-top2
#link = 'https://www.booking.com/attractions/it/pr9kidfuyv3x-venice-four-seasons-concert-in-the-vivaldi-church.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-132007&date=2024-03-24&start_time=19%3A00'
#save_file_path = "/Venice_Attraction_Reviews_Booking-top2.csv"
# Paris
# City Sightseeing Cruise
#link = 'https://www.booking.com/attractions/fr/prvpg5s3r713-city-sightseeing-cruise.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-1456928&date=2024-03-19&start_time=00%3A00&ticket_type=OF8RRb7XhLW7'
#save_file_path = "/Paris_Attraction_Reviews_Booking-top1.csv"
# Paris
# Admission to Disneyland® Paris
#link = 'https://www.booking.com/attractions/fr/prjyfitkuhfz-one-day-admission-to-disneylandr-paris.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&start_time=00%3A00&ufi=-1456928&date=2024-03-19'
#save_file_path = "/Paris_Attraction_Reviews_Booking-top2.csv"
#Ljubljana
#Admission to Ljubljana Castle-top1
#link = 'https://www.booking.com/attractions/si/prql2wwlkdmz-admission-to-ljubljana-castle.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-87271&date=2024-03-19&start_time=00%3A00'
#save_file_path = "/Ljubljana_Attraction_Reviews_Booking-top1.csv"
#Florance
#Sightseeing City Walking Tour - top1
#link = 'https://www.booking.com/attractions/it/prxbb82xigzq-sightseeing-city-walking-tour.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-117543&date=2024-03-19&start_time=10%3A00'
#save_file_path = "/Florance_Attraction_Reviews_Booking-top1.csv"
#Florance
#Accademia Gallery with Skip-the-line Entry - top2 - Non-extenable at Reviews
#link = 'https://www.booking.com/attractions/it/prcit3iikn7w-accademia-gallery-with-skip-the-line-entry.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-117543&date=2024-03-19&start_time=15%3A00&ticket_type=OFHJT65z0egB'
#save_file_path = save_file_path = "/Florance_Attraction_Reviews_Booking-top2.csv"

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path,firefox_binary_path = "C:/Program Files/Mozilla Firefox/firefox.exe")
browser.get(link)

# Click on Refuse Cookies
Refuse all cookies

In [15]:
refuse_cookies='//*[@id="onetrust-reject-all-handler"]'
check_and_click(browser, xpath = refuse_cookies, type = 'xpath')

In [16]:
# print average score
element = browser.find_element(By.CLASS_NAME, 'css-1n2v8f1')
print(link)
print(element.text)

https://www.booking.com/attractions/es/prp9at1egmyc-park-guell-admission-ticket.en-gb.html?label=en-es-booking-desktop-qYWoV_Fr%2AprpBspmirVxfgS652828998709%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp1005424%3Ali%3Adec%3Adm&aid=2311236&source=searchresults-product-card&ufi=-372490&date=2024-03-19&start_time=09%3A30&ticket_type=OFmQYxiImuT3
4.3 / 5
Very good
(1964 reviews)


# Click on Review Page
Click to open Review page

In [17]:
review_page='button.f88a5204c2:nth-child(3)'
check_and_click(browser, xpath = review_page, type = 'css')

# Click to sort by Newest First

In [18]:
sort_button='//*[@id="reviews-sort-option"]'
check_and_click(browser, xpath = sort_button, type = 'xpath')

In [19]:
Newest_first='/html/body/div[5]/div/div/div/div/div[1]/div[1]/h2/div/div/div[2]/div[2]/div[2]/div[2]/div/select/option[2]'
check_and_click(browser, xpath = Newest_first, type = 'xpath')

# Click to pick Number of Reviews in each page: 10 reviews -> 60 reviews

In [ ]:
'''
reviews_number_button='//*[@id=":r27:"]'
check_and_click(browser, xpath = reviews_number_button, type = 'xpath')
'''

In [20]:
select_page_number_60='/html/body/div[5]/div/div/div/div/div[2]/div/div[2]/div[2]/div/div[2]/div/select/option[4]'
check_and_click(browser, xpath = select_page_number_60, type = 'xpath')

# Scrapping Reviews

1. Nevigate to Review Setion in Selenium
2. Use Beautiful Soup to fetch and save in reviews variavle
3. loop to extrac review, date, name, rating, travel category, country, helpful_tag, translate_require
4. click to select next page (we ++index to loop over pages and pick page button by index)
5. -> repeat till end

In [21]:
# Expand reviews by click more button
def expand_Review_Contents(browser):
    """
    Expands the review contents by clicking the "Show More" buttons within each review.
    """
    # Wait for the reviews section to load
    try:
        WebDriverWait(browser, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, "dd5dccd82f"))
        )
    except TimeoutException:
        print("Timeout waiting for reviews section to load.")
        return
    # Find all reviews
    reviews = browser.find_elements(By.CLASS_NAME, "css-2opiv2")

    for review in reviews:
        try:
            show_more_button = browser.find_element(By.CSS_SELECTOR, '.css-m4qchi')
            if show_more_button is not None:
                check_and_click(browser,xpath=".css-m4qchi",type='css')
                time.sleep(2)
            else:
                pass

        except NoSuchElementException:
            # not found -> move to next review
            pass
        except TimeoutException:
            # time out
            print("Timeout clicking a 'Show More' button.")
            pass

# load page & return bs4 of reviews and page_bar
def load_Review_HTML(browser):
    # Navigate to the reviews section
    reviews_section = browser.find_element(By.CLASS_NAME, "dd5dccd82f")
    # Parse the HTML with BeautifulSoup
    reviews_section_html = reviews_section.get_attribute('outerHTML')
    soup = BeautifulSoup(reviews_section_html, 'html.parser')

    # All Reviews & Page_Bar
    reviews = soup.find_all('div', class_='css-2opiv2')
    page_bar_xpath = 'ab95b25344'
    page_bar = soup.find('div', class_= page_bar_xpath)
    return reviews, page_bar

# Last page number
def get_last_page_num(soup_page_bar):
    '''
    Get the number of pages. 
    '''
    a = soup_page_bar.find_all('li',class_="b16a89683f")
    return(int(a[-1].text))

# click the selected page of number
def click_page(browser, soup_page_bar, page_num):
    last_page_num = get_last_page_num(soup_page_bar)
    if page_num > last_page_num:
        print(f"Excess Page Num: {page_num}, Last Page: {last_page_num}")
        return False
    
    # page_bar in selenium
    reviews_section = browser.find_element(By.CLASS_NAME, "dd5dccd82f")
    page_bar_xpath = 'ab95b25344'
    page_bar = reviews_section.find_element(By.CLASS_NAME, page_bar_xpath)
    pages = page_bar.find_elements(By.CSS_SELECTOR, "li.b16a89683f")
    # Find clickable page button and Click
    try:
        for page in pages:
            # digital page.text & == page_num
            if page.text.isdigit() and (int(page.text) == page_num):
                print(page.get_attribute('outerHTML'))
                page.find_element(By.CSS_SELECTOR, ".a83ed08757.a2028338ea").click()
                break;
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        print(f"Failed to Click to Page: {page_num}")
        return False
    print(f"Success to Click to Page: {page_num}")
    return True

# write to csv, create if not-exist
#dfolder
def write_to_csv(reviews_df, save_path):

    # Check if the file exists
    file_exists = os.path.isfile(save_path)
    # If the file exists, append without writing the header; if not, write with the header.
    header = not file_exists  # True if file does not exist, hence write header
    mode = 'a' if file_exists else 'w'  # Append if file exists, else write
    
    # Write or append the DataFrame to the CSV file
    reviews_df.to_csv(save_path, mode=mode, header=header, index=False)
# Regex to select Date
def regex_Written_date(text):
    # Regex to select Date
    match = re.search(r"\d{1,2}\s\w+\s\d{4}", text)

    if match:
        date_str = match.group()
        date_obj = datetime.strptime(date_str, "%d %B %Y")
        formatted_date = date_obj.strftime("%Y-%m-%d")
    else:
        formatted_date = None
    return formatted_date
# Function to remove "Visited with " or "Visited " prefix
def remove_visited_prefix(text):
    return re.sub(r'^Visited (with )?', '', text)

################################################################################################################################################################
WebDriverWait(browser, 10).until(
    EC.visibility_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div"))
)

# setup page_num & initial write list
current_page = 1
review_write_list = []
for i in range(1,100):
    # click on show more button to expand reviews
    expand_Review_Contents(browser)
    # initializing reviews on each pages
    reviews,page_bar = load_Review_HTML(browser)

    last_page = get_last_page_num(page_bar)
    print(f"Current Page: {current_page}")
    print(f"Last Page: {last_page}")

    review_list = []  # Initialize empty list
    # Iterate over the found review divs
    for review in reviews:
        #review
        name = review.find("div", class_="a3332d346a")
        body = review.find("p", class_="a53cbfa6de css-16d87o3")
        #date
        written_date = review.find("div", class_="a53cbfa6de css-161x4ql")

        #country & travel_category 
        country_and_travel_category = review.find("div", class_="css-actu5n")
        if country_and_travel_category is not None:
            # select country which is direct child of class div.a53cbfa6de.f45d8e4c32
            country = country_and_travel_category.select("div.a53cbfa6de.f45d8e4c32")
            first_direct_child = None
            # Iterate through direct children
            for child in country_and_travel_category.children:
                # Check direct-child's Tag(div) and Class list
                if child.name == "div" and set(["a53cbfa6de", "f45d8e4c32"]).issubset(child.get('class', [])):
                    first_direct_child = child
                    break  # stop
            country = first_direct_child
            # select travel_category
            travel_category = country_and_travel_category.find("div", class_="css-1ltm3ye")
        else:
            # both country&travel_category is not recorded in review
            country = None
            travel_category = None 

        # rating, helpuful_tag, require translate to English
        rating = review.find("span", class_="a3332d346a")
        helpful_tag = review.find("button",class_="a83ed08757 c21c56c305 f38b6daa18 ab98298258 deab83296e bb803d8689")
        require_translate = review.find("a", class_="css-1riunrv")
        if require_translate is not None:
            print(require_translate.text)
            require_translate = True
        else:
            require_translate = False

        print("Reviewer name:",name.text)
        print("Review Body:",body.text.rstrip())# remove last white space \xa0
        print("Review Date:",regex_Written_date(written_date.text))
        print("Rating:",rating.text)
        print("Country :",country.text if country is not None else "None")
        print("Travel Category:",remove_visited_prefix(travel_category.text) if travel_category is not None else "None")
        print("Helpful_Tag:",helpful_tag.text if helpful_tag is not None else "None")
        print("Require Translate:", "Yes" if require_translate is True else "No")
        print("\n\n")

        # Then create a dictionary for the current review
        review_data = {
            'Reviewer_Name': name.text if name is not None else None,
            'Review_Body': body.text.rstrip(),
            'Review_Date': regex_Written_date(written_date.text),
            'Rating': rating.text if rating is not None else None,
            'Country': country.text if country is not None else None,
            'Helpful_Tag': helpful_tag.text if helpful_tag is not None else None,
            'Travel Category': remove_visited_prefix(travel_category.text) if travel_category is not None else None,
            'Require_Translate': require_translate
        }
        
        review_list.append(review_data)
        
    review_write_list.extend(review_list)
    # each 10 pages read into csv
    if current_page % 10 == 0:
        reviews_df = pd.DataFrame(review_write_list)
        display(reviews_df.head())
        write_to_csv(reviews_df, save_path = dfolder + save_file_path )
        review_write_list = []
        time.sleep(9)

    # Click Next Page
    is_clicking_last_page = click_page(browser= browser, soup_page_bar=page_bar, page_num= current_page+1)
    if is_clicking_last_page is not True: 
        reviews_df = pd.DataFrame(review_write_list)
        display(reviews_df.head())
        write_to_csv(reviews_df, save_path = dfolder + save_file_path )
        review_write_list = []
        time.sleep(15)
        browser.quit()

    current_page += 1
    # Wait Review Visible
    WebDriverWait(browser, 10).until(
        EC.visibility_of_element_located((By.XPATH, "/html/body/div[5]/div/div/div/div/div[2]/div/div[1]"))
    )
    time.sleep(2.5)

'''
# Iterate over the found review divs and process them as needed
for review in reviews:
    paragraphs = review.find_element(By.TAG_NAME, '//span[]')
    for p in paragraphs:
        print(p.text)
    # Example: Extracting the text of each review, adjust based on actual content
    review_text = review.text
    print(review_text)

# Don't forget to close the driver after you are done
#driver.quit()'''

Current Page: 1
Last Page: 33
Reviewer name: Ana
Review Body: Interesting place to visit, preferably in the morning 9 a.m. as the crowd on the stairs is bigger the later it gets (if you go in the morning go left at the entrance to go first to the stars and columns then go back and start over to catch everything) . The only down side is there is only a QR code with the map and if your internet is slow it is not very accessible, no printed version available.
Review Date: 2024-03-16
Rating: 5
Country : Serbia
Travel Category: family
Helpful_Tag: Helpful
Require Translate: No



Reviewer name: Anonymous
Review Body: had a hard time accessing the tickets at venue
Review Date: 2024-03-16
Rating: 4
Country : None
Travel Category: a partner or spouse
Helpful_Tag: Helpful
Require Translate: No



Reviewer name: Rivara
Review Body: There are no comments available for this review
Review Date: 2024-03-16
Rating: 4
Country : Italy
Travel Category: friends
Helpful_Tag: None
Require Translate: No



